In [14]:
import dao.MongoUsersSong as mus
import panda.SongUser as supd
# 展示操作一个用户的时候应该如何使用
mongomus = mus.UserSongRecom(user_id="1a78ac10b67911ed95dd00155dadb10b")
# mongomus.makeRecomUsersSetAnswer(limit="ONE")

In [15]:
# 获取recom中之前设置的数据
projections = {"_id": 0, "id": 1, "name": 1, "recomUsers": 1}
doc = mongomus.rdb.findDocument(
    collection_name="recom", query=mongomus.query, projection=projections)

# 储存请求用户的名字
mongomus.user_name = doc[0]['name']
# 跟据数据库给的RecUsers来处理数据
list_users_id = []
list_users_id.append(mongomus.user_id)
# print(self.user_id, list_users_id)
# users_id 合集，之后会通过这个来形成x轴坐标
list_users_id.extend(doc[0]['recomUsers'])
# 构成xy轴的内容源头
docs = mongomus.getSongsWithUserIdFromMongo(array=list_users_id)
# print(docs)

In [16]:
for item in doc[0].keys():
    print(item,":")
    print(doc[0][item])

id :
1a78ac10b67911ed95dd00155dadb10b
name :
RuRuUIH
recomUsers :
['1a78c6a0b67911ed95dd00155dadb10b', '1a78e1eeb67911ed95dd00155dadb10b', '1a78ba8eb67911ed95dd00155dadb10b', '1a78c308b67911ed95dd00155dadb10b', '1a78ee28b67911ed95dd00155dadb10b', '1a78b656b67911ed95dd00155dadb10b', '1a78d9f6b67911ed95dd00155dadb10b', '1a78bb2eb67911ed95dd00155dadb10b', '1a7904e4b67911ed95dd00155dadb10b', '1a78ba0cb67911ed95dd00155dadb10b', '1a7900f2b67911ed95dd00155dadb10b', '1a78d28ab67911ed95dd00155dadb10b', '1a790d2cb67911ed95dd00155dadb10b', '1a78e2b6b67911ed95dd00155dadb10b', '1a78bc78b67911ed95dd00155dadb10b', '1a791aceb67911ed95dd00155dadb10b', '1a78e130b67911ed95dd00155dadb10b', '1a790340b67911ed95dd00155dadb10b', '1a78b962b67911ed95dd00155dadb10b', '1a78d6f4b67911ed95dd00155dadb10b', '1a7909a8b67911ed95dd00155dadb10b', '1a78c22cb67911ed95dd00155dadb10b', '1a792000b67911ed95dd00155dadb10b', '1a791542b67911ed95dd00155dadb10b', '1a7922eeb67911ed95dd00155dadb10b', '1a78d046b67911ed95dd00155dadb10b

In [17]:
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongomus.makeXYMatric(docs=docs)
# print(data)
sudf = supd.SUPandas(datas=list(
    map(list, data.toarray())), items=mongomus.y, users=mongomus.x)
    

In [18]:
sudf.df

,RuRuUIH,zFpXHZ0n,m3NOW7Pp,G,Z,X2Bwh38,p,UnfVc1Tg,H6fLkSvrlg,fRcKBK16j,...,KvC,dyrvSYZxx,CO0y,BJ1HR0,D0nHKfvH,yNHtXCz5M,FnJi,afqmIx79w,dj775w,zAaptCORtu
567083018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29097995,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
29196302,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28377108,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
516358164,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377501157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2326503,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1540081,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400162805,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# 推荐过程
# dict_user_song = mongomus.makeRecomDcition(df_object=sudf, sign=-1)
similar = sudf.makeSimilarityBetweenUser()

/home/arki/.local/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [20]:
similar

,RuRuUIH,zFpXHZ0n,m3NOW7Pp,G,Z,X2Bwh38,p,UnfVc1Tg,H6fLkSvrlg,fRcKBK16j,...,KvC,dyrvSYZxx,CO0y,BJ1HR0,D0nHKfvH,yNHtXCz5M,FnJi,afqmIx79w,dj775w,zAaptCORtu
RuRuUIH,1.000000,0.014239,0.020774,0.010799,0.015355,0.015496,0.015343,0.017824,0.008159,0.014299,...,0.011655,0.006693,0.012694,0.007614,0.003448,0.007290,0.016895,0.007716,0.015831,0.011719
zFpXHZ0n,0.014239,1.000000,0.016468,0.010460,0.008341,0.014014,0.014938,0.009964,0.018203,0.011091,...,0.011261,0.012953,0.006729,0.017284,0.000000,0.010588,0.018462,0.010355,0.006289,0.022052
m3NOW7Pp,0.020774,0.016468,1.000000,0.013562,0.015523,0.012132,0.021841,0.023368,0.010587,0.012156,...,0.008621,0.010787,0.012332,0.009278,0.003927,0.007952,0.013239,0.007212,0.018124,0.017500
G,0.010799,0.010460,0.013562,1.000000,0.016667,0.018924,0.016579,0.015343,0.008919,0.022222,...,0.019080,0.008917,0.007979,0.010909,0.006472,0.009292,0.016211,0.013158,0.008717,0.012230
Z,0.015355,0.008341,0.015523,0.016667,1.000000,0.014172,0.019002,0.015472,0.011823,0.014901,...,0.011788,0.006608,0.012658,0.011640,0.004049,0.013292,0.010782,0.007417,0.014146,0.016115
X2Bwh38,0.015496,0.014014,0.012132,0.018924,0.014172,1.000000,0.016779,0.011217,0.013845,0.012313,...,0.010582,0.008413,0.006250,0.011481,0.013636,0.014396,0.012536,0.006803,0.009423,0.011712
p,0.015343,0.014938,0.021841,0.016579,0.019002,0.016779,1.000000,0.018605,0.012026,0.019716,...,0.016682,0.017635,0.007447,0.011869,0.004957,0.015355,0.017933,0.006849,0.014213,0.019339
UnfVc1Tg,0.017824,0.009964,0.023368,0.015343,0.015472,0.011217,0.018605,1.000000,0.016393,0.017872,...,0.009551,0.015102,0.017957,0.012358,0.006527,0.005929,0.015873,0.010804,0.020213,0.021685
H6fLkSvrlg,0.008159,0.018203,0.010587,0.008919,0.011823,0.013845,0.012026,0.016393,1.000000,0.014735,...,0.009650,0.011220,0.014771,0.006596,0.009158,0.010127,0.009751,0.001610,0.008186,0.013105
fRcKBK16j,0.014299,0.011091,0.012156,0.022222,0.014901,0.012313,0.019716,0.017872,0.014735,1.000000,...,0.010732,0.008772,0.004535,0.013699,0.005362,0.009109,0.013441,0.007362,0.008602,0.018613


In [21]:
# 使用平均数，对以上的相似度结果求平均，并取高于平均数的对象形成一个list
diction_user = sudf.makeTopNUsers(similar=similar,sign=1)
print(diction_user)
diction = sudf.makeRecomUserBySong(
            topN=diction_user, song_num=50)

{'RuRuUIH': ['HY0HwB', 'm3NOW7Pp', 'axRJKQiq5', 'UnfVc1Tg', 'FnJi', 'GOUkTpak', 'dj775w', 'X2Bwh38', 'Z', 'p', 'J94COPc', '1PufH', 'fRcKBK16j', 'S0JlP3z', 'zFpXHZ0n', 'HrlIEqGS', 'b', 'CO0y'], 'zFpXHZ0n': ['zAaptCORtu', 'FnJi', 'H6fLkSvrlg', 'BJ1HR0', 'm3NOW7Pp', 'HY0HwB', 'p', 'N', 'GOUkTpak', 'RuRuUIH', 'X2Bwh38', 'b', 'dyrvSYZxx', 'axRJKQiq5', '79vexmSD', '1PufH', '6p11UivDhF', 'KvC'], 'm3NOW7Pp': ['S0JlP3z', 'UnfVc1Tg', 'p', 'RuRuUIH', 'N', 'GOUkTpak', 'dj775w', 'zAaptCORtu', '1PufH', 'zFpXHZ0n', 'Z', 'HY0HwB', 'b', '6p11UivDhF', 'HrlIEqGS', 'J94COPc', 'uTAbZm'], 'G': ['fRcKBK16j', '6p11UivDhF', 'S0JlP3z', 'KvC', 'X2Bwh38', 'Z', 'p', 'FnJi', 'GOUkTpak', 'BK1076n', 'UnfVc1Tg', 'I3F0KrG3a', 'ere', '9ZxdOqP', 'm3NOW7Pp', 'afqmIx79w', 'J94COPc'], 'Z': ['1PufH', 'p', 'uTAbZm', 'G', 'zAaptCORtu', 'GOUkTpak', 'm3NOW7Pp', 'UnfVc1Tg', 'RuRuUIH', 'fRcKBK16j', 'S0JlP3z', 'J94COPc', 'X2Bwh38', 'dj775w', 'yNHtXCz5M', 'CO0y'], 'X2Bwh38': ['GOUkTpak', '9ZxdOqP', 'G', '6p11UivDhF', 'J94COPc', 'p',

In [22]:
# 程序过程中会有sign 来判断是去单个，还是返回所有的
dict_result = {mongomus.user_name: [diction[mongomus.user_name],diction_user[mongomus.user_name]]}
print(dict_result)

{'RuRuUIH': [[2867202, 1330364421, 567083018, 29097995, 1309261838, 28377108, 29999126, 40255512, 39436317, 1456603168, 31965219, 25657383, 26116136, 1605671, 524331, 1830469676, 426852405, 622645, 1490997, 23986232, 31260730, 41631802, 27836485, 425295943, 545407049, 479281226, 545407050, 19038288, 1495384148, 1355169880, 39436376, 360539, 1303019613, 344157, 1490124895, 501514336, 471613535, 2195553, 16466019, 1949795, 5046372, 501514341, 4128869, 5193833, 400162922, 400162923, 464683120, 5193843, 1456537722, 1397293188], ['HY0HwB', 'm3NOW7Pp', 'axRJKQiq5', 'UnfVc1Tg', 'FnJi', 'GOUkTpak', 'dj775w', 'X2Bwh38', 'Z', 'p', 'J94COPc', '1PufH', 'fRcKBK16j', 'S0JlP3z', 'zFpXHZ0n', 'HrlIEqGS', 'b', 'CO0y']]}


In [23]:
print(dict_result)

{'RuRuUIH': [[2867202, 1330364421, 567083018, 29097995, 1309261838, 28377108, 29999126, 40255512, 39436317, 1456603168, 31965219, 25657383, 26116136, 1605671, 524331, 1830469676, 426852405, 622645, 1490997, 23986232, 31260730, 41631802, 27836485, 425295943, 545407049, 479281226, 545407050, 19038288, 1495384148, 1355169880, 39436376, 360539, 1303019613, 344157, 1490124895, 501514336, 471613535, 2195553, 16466019, 1949795, 5046372, 501514341, 4128869, 5193833, 400162922, 400162923, 464683120, 5193843, 1456537722, 1397293188], ['HY0HwB', 'm3NOW7Pp', 'axRJKQiq5', 'UnfVc1Tg', 'FnJi', 'GOUkTpak', 'dj775w', 'X2Bwh38', 'Z', 'p', 'J94COPc', '1PufH', 'fRcKBK16j', 'S0JlP3z', 'zFpXHZ0n', 'HrlIEqGS', 'b', 'CO0y']]}


In [24]:
# 最后保存
# mongomus.saveUserSongRecomAnswer(diction=dict_result)
docs = mongomus.changeDataFormat(diction=dict_result, list_keys=list(dict_result))
# print(docs)
mongomus.wdb.writeDocument(
    docs=docs, collection_name="recom")


update coll RuRuUIH in recom
